In [28]:
# !pip3 install --upgrade --quiet  langchain langchain-community langchainhub gpt4all chromadb 
# !pip3 install unstructured
# !pip3 install sentence-transformers

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.embeddings import JinaEmbeddings


import os

In [29]:
directories = ["Web Scholar PDFs", "Data/About Scottie", "Data/Buggy News", "academic_calendars", "Data/history_of_cmu", "Data/history_of_scs", "Data/Kiltie Band", "Data/lti_faculty", "Data/lti_programs", "program_handbooks", "Data/Tartan Facts"]


In [ ]:
docs = []
for i in directories:
    loader = DirectoryLoader(i, glob="*", exclude="annotation.txt", show_progress=True)
    docs.append(loader.load())

In [ ]:
all_splits = []
for i in docs:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
    all_splits.append(text_splitter.split_documents(i))

In [30]:
import pickle as pkl

# with open('noWebScholarSplitDocuments.pkl', "wb") as f:
#     pkl.dump(all_splits[1:], f)


with open('splitDocuments.pkl','rb') as f: 
  all_splits = pkl.load(f)

In [10]:
# 1, 2, 6

preciseSplits = [all_splits[i] for i in [1,2,6]] #scotty, mascot, terrier, scottish
nonPrecise = [all_splits[i] for i in range(len(all_splits)) if i not in [1,2,6]]

In [32]:
from langchain.embeddings import HuggingFaceEmbeddings


# model_name = "Salesforce/SFR-Embedding-Mistral"
# model_kwargs = {"device": "cuda"}
# embedding_function = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

# import chromadb.utils.embedding_functions as embedding_functions
# huggingface_ef = embedding_functions.HuggingFaceEmbeddingFunction(
#     model_name="BAAI/llm-embedder"
# )

model_name = "BAAI/llm-embedder"
model_kwargs = {"device": "cpu"}
embedding_function = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)


# embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

for i in all_splits:
    vectorstore = Chroma.from_documents(documents=i, embedding=embedding_function, persist_directory="llm-embedder/")

KeyboardInterrupt: 

In [15]:
for i in nonPrecise:
    vectorstore = Chroma.from_documents(documents=i, embedding=embedding_function, persist_directory="nonPrecise/")

In [16]:
vectorstore = Chroma(persist_directory="nonPrecise", embedding_function=embedding_function)

In [ ]:
vectorstore.similarity_search("")

In [26]:
import pickle as pkl
from langchain.retrievers import BM25Retriever

with open('splitDocuments.pkl','rb') as f:
  all_splits = pkl.load(f)

def flatten_extend(matrix):
    flat_list = []
    for row in matrix:
        flat_list.extend(row)
    return flat_list


bm25_retriever = BM25Retriever.from_documents(flatten_extend(all_splits))

In [27]:
bm25_retriever.get_relevant_documents("What is the purpose of the ACL 60/60 evaluation sets?")

[Document(page_content='What is classification? What is its purpose and its impact in different institutional contexts? What can those impacts add up to in our material realities? Sorting Sexualities encourages us to ask these questions when we think about and describe sexuality. This slim but dense volume delves deeply into the ways that sexuality is seen and understood in different legal settings. In doing so, it leads the reader to confront the contradic- tion between our common social understanding of sexuality as', metadata={'source': 'Web Scholar PDFs/eea8803365fcbfdca8ba9ef0ec4e90cfd7620bf2.pdf'}),
 Document(page_content='In the experiment section, we aim to answer the following research questions: 1) What is the per- formance across various tasks within the English language? 2) What is the performance across dif- ferent domains within the same task? 3) What is the performance across different evaluation dimen- sions? 4) What is the performance at unseen tasks? 5) Given that LLa